In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

from glob import iglob
from math import log, floor
import os
import re
import pickle

%reload_ext autoreload
%load_ext blackcellmagic
%autoreload 2

from lib.plot_utils import *
from lib.cdf_utils import *

# POP

## Private WAN

In [ ]:
TIME_WINDOW = 5
fnames = list(iglob("private_wan_csvs/*.csv"))
time_cols = ["tm_date", "tm_time"]
data_cols = ["nc_time", "pop_time", "nc_flow", "pop_flow"]
cols = time_cols + data_cols

def add_timestamp_col(df):
    df["timestamp"] = pd.to_datetime(
        df["tm_date"] + " " + df["tm_time"].map(lambda x: x[:-4]),
        format="%Y%m%d %H%M%S",
    )
    df["timedelta"] = np.arange(0, len(df) * TIME_WINDOW, TIME_WINDOW) / (24 * 60)
    return df.drop(columns=time_cols)


def add_split_threshold_suffix(df, split_threshold):
    return df.rename(
        {col: "{}_{}".format(col, split_threshold) for col in data_cols},
        axis=1,
    )


dfs = [
    add_split_threshold_suffix(df, split_threshold)
    for split_threshold, df in [
        [
            float(re.search("split_threshold=(.*)-sanitized", fname).group(1)),
            pd.read_csv(fname, dtype={"tm_date": str, "tm_time": str}, usecols=cols),
        ]
        for fname in fnames
    ]
]

join_df = dfs[0]
index_cols = ['tm_date', 'tm_time']
for other_df in dfs[1:]:
    join_df = join_df.set_index(index_cols).join(other_df.set_index(index_cols)).reset_index()

ready_to_plot_df = add_timestamp_col(join_df)

In [ ]:
print(ready_to_plot_df['pop_flow_0.0'].median())
print(ready_to_plot_df['pop_flow_0.25'].median())
print(1/ ready_to_plot_df['pop_time_0.0'].median())
print(1/ ready_to_plot_df['pop_time_0.25'].median())

In [ ]:
MA_WINDOW = 5
MARK_EVERY = 150
MARKER_SIZE = 10
LINE_WIDTH = 2.0

def moving_average(a, n=MA_WINDOW):
    a = np.array(a)
    ret = np.cumsum(a)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1 :] / n


fig, [ax0, ax1] = plt.subplots(nrows=2, ncols=1, figsize=(8.38, 6.5), sharex=True)
ax0.plot(
    ready_to_plot_df["timedelta"][MA_WINDOW - 1 :],
    moving_average(ready_to_plot_df["nc_flow_0.0"]),
    # marker = 's',
    # markersize = MARKER_SIZE,
    # markevery = MARK_EVERY,
    linestyle= '--',
    linewidth = LINE_WIDTH,
    label="NCFlow",
)
ax0.plot(
    ready_to_plot_df["timedelta"][MA_WINDOW - 1 :],
    moving_average(ready_to_plot_df["pop_flow_0.0"]),
    # marker = 'o',
    # markersize = MARKER_SIZE,
    # markevery = MARK_EVERY,
    linestyle= ':',
    linewidth = LINE_WIDTH,
    label="POP, +0x",
)
ax0.plot(
    ready_to_plot_df["timedelta"][MA_WINDOW - 1 :],
    moving_average(ready_to_plot_df["pop_flow_0.25"]),
    # marker = '^',
    # markersize = MARKER_SIZE,
    # markevery = MARK_EVERY,
    linestyle= '-',
    linewidth = LINE_WIDTH,
    label="POP, +0.25x",
)
ax0.set_ylabel("Total flow rel. to\n original problem")
ax0.grid(True)

ax1.plot(
    ready_to_plot_df["timedelta"][MA_WINDOW - 1 :],
    moving_average(1 / ready_to_plot_df["nc_time_0.0"]),
    # marker = 's',
    # markersize = MARKER_SIZE,
    # markevery = MARK_EVERY,
    linestyle= '--',
    linewidth = LINE_WIDTH,
    label="NCFlow",
)
ax1.plot(
    ready_to_plot_df["timedelta"][MA_WINDOW - 1 :],
    moving_average(1 / ready_to_plot_df["pop_time_0.0"]),
    # marker = 'o',
    # markersize = MARKER_SIZE,
    # markevery = MARK_EVERY,
    linestyle= ':',
    linewidth = LINE_WIDTH,
    label="POP, +0x",
)
ax1.plot(
    ready_to_plot_df["timedelta"][MA_WINDOW - 1 :],
    moving_average(1 / ready_to_plot_df["pop_time_0.25"]),
    # marker = '^',
    # markersize = MARKER_SIZE,
    # markevery = MARK_EVERY,
    linestyle= '-',
    linewidth = LINE_WIDTH,
    label="POP, +0.25x",
)
ax1.set_xlabel("Time (days)")
ax1.set_ylabel("Speedup rel. to\n original problem")
ax1.grid(True)

legend = ax0.legend(
    frameon=False, ncol=3, loc="center", bbox_to_anchor=(0, 0, 1.0, 2.3)
)
save_figure("private-wan", extra_artists=(legend,))

## Maximum Concurrent Flow

In [ ]:
# Path Formulation DF
path_form_df = (
    pd.read_csv("scripts/mcf_cdfs/path-form.csv")
    .drop(columns=["num_nodes", "num_edges", "num_commodities"])
    .query(PF_PARAMS)
)
path_form_df = sort_and_set_index(path_form_df, drop=True)

In [ ]:
# POP DF
pop_df = pd.read_csv("scripts/mcf_cdfs/pop.csv")
pop_df = sort_and_set_index(pop_df, drop=True)

In [ ]:
pop_mcf_ratio_df = get_ratio_df(pop_df, path_form_df, "obj_val", "_pop")

In [ ]:
pop_mcf_ratio_df.query("problem == 'Kdl.graphml'").sort_values("obj_val_ratio").head(20)

### CDFs

In [ ]:
from mcf_cdfs.plot_mcf_cdfs import plot_mcf_cdfs
plot_mcf_cdfs('mcf_cdfs/', title='Poisson High Intra', query_str='problem not in ["Uninett2010.graphml", "Ion.graphml", "Interoute.graphml"] and tm_model == "poisson-high-intra"')

In [ ]:
from mcf_cdfs.plot_mcf_cdfs import plot_mcf_cdfs
plot_mcf_cdfs('mcf_cdfs/', title='Uniform', query_str='problem not in ["Uninett2010.graphml", "Ion.graphml", "Interoute.graphml"] and tm_model == "uniform"')

## Max Flow

### CDFs

In [ ]:
from speedup_relative_flow_cdfs.plot_speedup_relative_flow_cdfs import plot_speedup_relative_flow_cdfs
plot_speedup_relative_flow_cdfs('speedup_relative_flow_cdfs/', title='Uniform', query_str='problem not in ["Uninett2010.graphml", "Ion.graphml", "Interoute.graphml"] and tm_model == "uniform"')

In [ ]:
plot_speedup_relative_flow_cdfs('speedup_relative_flow_cdfs/', title='Gravity', query_str='problem not in ["Uninett2010.graphml", "Ion.graphml", "Interoute.graphml"] and tm_model == "gravity"')

In [ ]:
plot_speedup_relative_flow_cdfs('speedup_relative_flow_cdfs/', title='Bimodal', query_str='problem not in ["Uninett2010.graphml", "Ion.graphml", "Interoute.graphml"] and tm_model == "bimodal"')

In [ ]:
plot_speedup_relative_flow_cdfs('speedup_relative_flow_cdfs/', title='Poisson High Inter', query_str='problem not in ["Uninett2010.graphml", "Ion.graphml", "Interoute.graphml"] and tm_model == "poisson-high-inter"')

In [ ]:
plot_speedup_relative_flow_cdfs('speedup_relative_flow_cdfs/', title='Poisson High Intra', query_str='problem not in ["Uninett2010.graphml", "Ion.graphml", "Interoute.graphml"] and tm_model == "poisson-high-intra"')

In [ ]:
plot_speedup_relative_flow_cdfs('speedup_relative_flow_cdfs/', title='Kdl', query_str='problem not in ["Uninett2010.graphml", "Ion.graphml", "Interoute.graphml"] and problem == "Kdl.graphml"')

In [ ]:
plot_speedup_relative_flow_cdfs('speedup_relative_flow_cdfs/', title='GtsCe', query_str='problem not in ["Uninett2010.graphml", "Ion.graphml", "Interoute.graphml"] and problem == "GtsCe.graphml"')

In [ ]:
plot_speedup_relative_flow_cdfs('speedup_relative_flow_cdfs/', title=r'$\alpha \in \{1, 2\}$', query_str='problem not in ["Uninett2010.graphml", "Ion.graphml", "Interoute.graphml"] and scale_factor in [1.0, 2.0]')

In [ ]:
plot_speedup_relative_flow_cdfs('speedup_relative_flow_cdfs/', title=r'$\alpha$ = 16', query_str='problem not in ["Uninett2010.graphml", "Ion.graphml", "Interoute.graphml"] and scale_factor == 16.0')

In [ ]:
plot_speedup_relative_flow_cdfs('speedup_relative_flow_cdfs/', title=r'$\alpha$ = 128', query_str='problem not in ["Uninett2010.graphml", "Ion.graphml", "Interoute.graphml"] and scale_factor == 128.0')

## Scatter Plots

In [ ]:
from scatter_plot.plot_scatter_plot import scatter

In [ ]:
def pop_random_scatter(query_str, title):
    plot_dfs = [df.query(query_str) for df in ratio_dfs[0:3]]
    scatter(plot_dfs,
            ["POP: Random, 64", "POP: Random, 16", "POP: Random, 4"],
            ["nc", "smore", "pf"],
            "flow_ratio",
            "speedup_ratio",
            xlim=None,
            ylim=None,
            xlabel='flow ratio to PF4',
            ylabel='speedup ratio to PF4',
            bbta=(0, 0, 1, 2.35),
            ncol=3,
            title=title,
            log=True) 

In [ ]:
pop_random_scatter("tm_model == 'poisson-high-intra'", "Poisson, High Intra")


In [ ]:
pop_random_scatter("tm_model == 'poisson-high-inter'", "Poisson, High Inter")

In [ ]:
pop_random_scatter("tm_model == 'uniform'", "Uniform")

In [ ]:
pop_random_scatter("tm_model == 'gravity'", "Gravity")

In [ ]:
pop_random_scatter("tm_model == 'bimodal'", "Bimodal")